In [4]:
import numpy as np
import random
import time

In [21]:
from IPython.display import clear_output

### CREATE A SNAKE GAME

In [561]:
class create_game:
    def __init__(self, dimensions):
        
        # CREATE A MAZE, SNAKE & FOOD
        self.create_maze(dimensions)
        self.create_snake()
        self.create_food()
        
        # START AT SCORE ZERO
        self.score = 0
        
        # SHOW THE CURRENT MAZE
        self.show()
    
    # SHOW THE CURRENT MAZE
    def show(self):
        clear_output(wait=True)
        
        # LOOP THROUGH MAZE ROWS & COLS
        for row in self.maze:
            for index, value in enumerate(row):

                # SELECT CORRECT SYMBOL
                if value == 0:
                    symbol = '-'
                elif value == 1:
                    symbol = '#'
                elif value == 2:
                    symbol = '@'

                # PRINT WITH OR WITHOUT LINEBREAK
                if index < len(row) - 1:
                    print(symbol, end='')
                else:
                    print(symbol, end='\n')
        
    # END THE GAME
    def destroy(self):
        clear_output(wait=True)
        print('YOU HAVE DIED - SCORE: {}'.format(self.score))
        
    # CREATE A MAZE
    def create_maze(self, dimensions):
        self.rows = dimensions[0]
        self.cols = dimensions[1]
        self.maze = np.zeros((dimensions[0], dimensions[1]))
        
    # CREATE FOOD AT A RANDOM POSITION
    def create_food(self):
        
        # FIND ALL EMPTY POSITIONS IN THE MATRIX
        positions = np.argwhere(self.maze == 0)
        
        # PICK A RANDOM OPEN INDEX
        index = random.randint(0, len(positions) - 1)
        
        # ADD THE FOOD & UPDATE THE FIGURE
        self.maze[positions[index][0]][positions[index][1]] = 2
    
    # CREATE A SNAKE
    def create_snake(self):
        self.snake = [(1, 3), (1, 2), (1, 1)]
        
        # LOOP THROUGH BODYPARTS & ADD THEM
        for part in self.snake:
            self.maze[part[0]][part[1]] = 1
        
    # MOVE THE SNAKE
    def move_snake(self, direction):

        # SNAPSHOT CURRENT HEAD POSITION
        next_y = self.snake[0][0]
        next_x = self.snake[0][1]

        # DECIDE THE NEXT MOVE
        if direction == 'DOWN':
            next_y += 1

        elif direction == 'UP':
            next_y -= 1

        elif direction == 'RIGHT':
            next_x += 1

        elif direction == 'LEFT':
            next_x -= 1

        # THE THE SNAKE HITS ITSELF
        if ((next_y, next_x) in self.snake):
            self.destroy()

        # IF THE UPPER OR LOWER WALL
        elif (next_y < 0 or next_y > self.rows - 1):
            self.destroy()

        # IF THE LEFT OR RIGHT WALL
        elif (next_x < 0 or next_x > self.cols - 1):
            self.destroy()

        # EAT FOOD & GROW
        elif (self.maze[next_y][next_x] == 2):

            # GROW THE SNAKE BY UPDATING THE HEADS POSITION
            self.snake.insert(0, (next_y, next_x))
            self.maze[next_y][next_x] = 1

            # INCREASE THE SCORE & CREATE NEW FOOD
            self.score += 1
            self.create_food()

            # SHOW UPDATED MAZE
            self.show()

        # OTHERWISE, MOVE
        else:

            # MOVE SNAKE HEAD
            self.snake.insert(0, (next_y, next_x))

            # REMOVE SNAKE TAIL
            tail = self.snake.pop()

            # RENDER NEW HEAD & REMOVE OLD TAIL
            self.maze[next_y][next_x] = 1
            self.maze[tail[0]][tail[1]] = 0

            # SHOW UPDATED MAZE
            self.show()

### PLAY THE GAME

In [562]:
game = create_game([10, 20])

--------------------
-###----------------
--------------------
--------------------
--------------------
--------@-----------
--------------------
--------------------
--------------------
--------------------


In [577]:
game.move_snake('LEFT')

-----------------@--
--------------------
--------------------
--------------------
--------------------
--------------------
--------#-----------
-----####-----------
--------------------
--------------------


In [565]:
#for direction in ['RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT', 'RIGHT']:
#    game.move_snake(direction)
#    time.sleep(1)